In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile

# Directory containing the zip files
directory = r'C:\AIS_ZIP\2024'

# URL to check for updates
url = 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2024/index.html'

# Get the list of existing zip files in the directory
existing_files = set(os.listdir(directory))

# Fetch the HTML content from the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Get the list of zip files from the HTML content
remote_files = set(link['href'].split('/')[-1] for link in soup.find_all('a', href=True) if link['href'].endswith('.zip'))

# Determine which files need to be downloaded (new or updated)
files_to_download = remote_files - existing_files

print(files_to_download)
print(f"Number of files to download: {len(files_to_download)}")

for file_name in files_to_download:
    zip_path = os.path.join(directory, file_name)
    zip_url = url.rsplit('/', 1)[0] + '/' + file_name
    zip_response = requests.get(zip_url)
    with open(zip_path, 'wb') as f:
        f.write(zip_response.content)
    print(f'Downloaded: {file_name}')
    
    # Extract the downloaded zip file
    extract_directory = r'C:\AIS\AIS_2024'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_directory)
    print(f'Extracted: {file_name} to {extract_directory}')